In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import getpass
import folium
import nivapy
from sqlalchemy import create_engine

# NOPE metals

We need to develop an extension module for NOPE to simulate metals. It's not clear yet exactly how this should be done, but the first step is to explore some of the datasets sent by Øyvind following our meeting in Grimstad in September.

## 1. Data exploration

### 1.1. Data from Øyvind

#### 1.1.1. Spatial data

One of the raw datasets is a little unusual. It was originally sent to Øyvind by Anders Finstad and then forwarded on to me (see e-mail from Øyvind received 28/09/2017 at 15.52). The file is called *ecco_biwa_db_storage.csv* and it's large (nearly 700 MB). Anders hasn't provided much background information, but it looks as though this is a direct dump from a spatial database and the file includes hexadecimal-encoded Well-Known Binary (WKB) spatial data.

**Note:** The original file includes duplicated column headings `geom` and `ebint` (it looks as though this is due to a previous spatial join). I have therefore renamed the first occurrences to `geom2` and `ebint2`, respectively, to avoid naming conflicts.

The file contains a huge amount of data (nearly 500 columns), so it'll take a bit of figuring out. I also need to parse the hex-encoded WKB into something that I can display on a map.

In [2]:
# Read raw data
in_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
          r'\NOPE\Metals\Raw_Datasets\ecco_biwa_db_storage.csv')
df = pd.read_csv(in_csv, sep=';')

# Tidy geom cols
df['geom'] = df['geom2']
del df['geom2'], df['ebint'], df['ebint2']

df.head()

,corine_2000_continuous_urban_fabric_area_km2,corine_2000_discontinuous_urban_fabric_area_km2,corine_2000_industrial_or_commercial_units_area_km2,corine_2000_road_and_rail_networks_and_associated_land_area_km2,corine_2000_port_areas_area_km2,corine_2000_airports_area_km2,corine_2000_mineral_extraction_sites_area_km2,corine_2000_dump_sites_area_km2,corine_2000_construction_sites_area_km2,corine_2000_green_urban_areas_area_km2,...,a_,column_44,column_45,no3_ug_l,no3_p_vektbasert,dist_closest_ebint,no_lakes_in_250m,no_lakes_in_100m,no_lakes_in_10m,dist_2nd_closest_ebint
0,0,0,0,0,0,0,0,0,0,0,...,82.414563,82.000000,NaN,248.020800,0.396833,23.216747,1,1,0,NaN
1,0,0,0,0,0,0,0,0,0,0,...,48.164530,83.992481,46.80,230.305029,0.397763,0.000000,1,1,1,NaN
2,0,0,0,0,0,0,0,0,0,0,...,58.657574,47.000000,NaN,62.005200,0.169877,0.000000,1,1,1,NaN
3,0,0,0,0,0,0,0,0,0,0,...,85.204968,52.000000,12.92,105.408840,0.215120,0.000000,1,1,1,NaN
4,0,0,0,0,0,0,0,0,0,0,...,55.653402,154.259399,6.75,44.289429,0.150644,0.000000,1,1,1,NaN


It is possible to use [Shapely](https://github.com/Toblerity/Shapely) to parse the WKB data, which converts the dataframe to a geodataframe. My intial idea was to then save this as a shapefile for display in ArcGIS. However, the column names in the file are too long for the shapefile format, and naming conflicts arise when the columns are truncated. An alternative is therefore to save the data in GeoJSON format, which is more flexible. Unfortunately, there's then no way to read this using ArcMap. QGIS can read GeoJSON, but a 700 MB JSON file is unwieldy and difficult to manipulate. The code below is nevertheless useful and worth recording for the future.

In [3]:
#from shapely import wkb
#from functools import partial # Makes it possible to use "map" with kwargs - see
#                              # https://stackoverflow.com/a/13499853/505698
#
## We need to apply wkb.loads to the 'geom2' col, using the kwarg
## hex=True. To do this, use 'partial' to create a new func
#map_func = partial(wkb.loads, hex=True)
#
## Parse geometry data
#geometry = df['geom2'].map(map_func)
#
## Delete plain text geom
#df = df.drop('geom2', axis=1)
#
## Build gdf
#crs = {'init': 'epsg:32633'} # Numbers look like UTM. Assume Zone 33N
#gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
#
## Write to GeoJSON
## Works, but some encoding issues with special chars in some columns
## Don't think utf-8 is supported
#out_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
#            r'\NOPE\Metals\Raw_Datasets\ecco_biwa_db_storage.json')
#gdf.to_file(out_path, driver='GeoJSON')

Rather than working with a huge GeoJSON file, it would be better to put this information back into a spatial database. I've spent a few hours messing around with Spatialite, but the development is patchy and there are issues with getting everything installed correctly. Although it seems a bit over-the-top, it's actually much easier to just install PostGIS.

Having [installed PostgreSQL 9.6 and PostGIS 2.4](http://www.bostongis.com/PrinterFriendly.aspx?content_name=postgis_tut01), I've created a new database called `niva_work`, which I'll use for manipulating spatial datasets in the future. For reference, having created this database, spatial extensions are enabled by right-clicking `Extensions` and choosing `PostGIS` from the list of extension names. I have then created a new schema called `nope_metals`, which I'll use for this project.

I have also done some further development of NivaPy to make it easier to integrate spatial and non-spatial data processing in different databases - see this [notebook](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/oracle_postgis_test.ipynb) for details.

In [4]:
# Connect to db
pg_eng = nivapy.da.connect(src='postgres')

Username: ········
Password: ········
Connection successful.


In [5]:
# Write raw text data to new table
df.to_sql('ecco_biwa', schema='nope_metals', index=True, 
          con=pg_eng, if_exists='replace')

# Parse geom column from hex-encoded WKB. See:
# https://gis.stackexchange.com/a/233215/2131
sql = ("ALTER TABLE niva_work.nope_metals.ecco_biwa "
       "ALTER COLUMN geom TYPE geometry(MULTIPOLYGON, 25833) " # UTM Zone 33N ETRS89
       "USING ST_SetSRID(geom, 25833)")
res = pg_eng.execute(sql)

# Build spatial index
sql = ("CREATE INDEX nope_metals_ecco_biwa_gix "
       "ON niva_work.nope_metals.ecco_biwa "
       "USING GIST (geom)")
res = pg_eng.execute(sql)

This data can now be explored using QGIS (`Layer > Add layers > Add PostGIS layers`) by supplying the following credentials:

    Name:     niva_work
    Host:     localhost
    Database: niva_work
    
The CSV has data for 4677 catchments across Fennoscandia, of which 990 are located in Norway. As demonstrated in this [notebook](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/oracle_postgis_test.ipynb), I'm pretty sure this is the "1000 lakes" dataset, where each polygon corresponds to a lake catchment area. A large number of parameters have been derived for each location, including air and water chemistry (but not much on metals), CORINE and NDVI statistics over time, population data etc.

#### 1.1.2. Other datasets

In addition to the spatial dataset described above, Øyvind has also supplied the following (via Tom Andersen):

 * **N1k_dat_29102014.txt**. Catchment characteristics for the "1000 lakes" survey
 
 * **ReginCC_data_170708b.txt**. Catchment properties for around 8000 regine catchments
 
Both these datasets are potentially useful, but it is difficult to interpret exactly what they mean without more details regarding column headings etc. **Come back to this later**.

### 1.2. External datasets

The [Meteorological Synthesizing Centre-East (MSC-E)](http://www.msceast.org/index.php/pollution-assessment/emep-domain-menu/data-hm-pop-menu) is part of EMEP, and their website includes simulated deposition of some metals (Cd, Hg and Pb) for Norway in recent years (2014 and 2015). The data are provided on the 50 km EMEP grid and are saved locally here:

C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet\NOPE\Metals\Raw_Datasets\EMEP


 
## 2. "1000 lakes" dataset

Before exploring the datastes above in too much detail, I'd like to investigate the data we already have in RESA2. In 1995, NIVA undertook a national scale survey of 1000 lakes, which included testing for metals

This seems like a good starting point. As an initial data exploration, I'll attempt the following workflow:

  1. Extract metal concentrations (Ag, As, Pb, Cd, Cu, Cr, Ni, Hg and Zn) from RESA2 for all the lakes in the 1995 "1000 lakes" survey <br><br>
  
  2. Link this data to e.g. atmosphereic deposition, land use, populations etc. in the spatial data from Anders  <br><br>
  
  3. See if any relationships can be identified from which national scale metals loads can be estimated

## 1. "1000 lakes" water chemistry

**(This is copied from another notebook as it's useful, but no longer relevant at the original location).**

The first step is to extract water chemistry data for the 1995 "1000 lakes" survey. Note that Hg wasn't measured during this survey, but most other metals of interest were. I also want to extract data for pH and TOC if this is available, as these factors affect the mobility of metals from the soil to the water.

In [ ]:
# Search projects
prj_df = nivapy.da.search_projects("project_name LIKE 'Regional%'", ora_eng)
prj_df

In [ ]:
# Get stations for 1995
stn_df = nivapy.da.select_project_stations([27,], ora_eng)

print 'Number of stations:', len(stn_df)
stn_df.head()

In [ ]:
# Pars of interest
par_list = ['Ag', 'As', 'Pb', 'Cd', 'Cu', 'Zn', 'Ni', 'Cr', 'pH', 'TOC']

# Container for results
df_list = []

# Loop over stations
for stn_id in stn_df['station_id']:
    # Get all Al data
    wc_df, dup_df = nivapy.da.extract_water_chem(stn_id, par_list,
                                                 '1995-01-01', 
                                                 '1995-12-31',
                                                 ora_eng,
                                                 plot=False)
    
    # Tidy
    wc_df['station_id'] = stn_id
    wc_df.reset_index(inplace=True)
    df_list.append(wc_df)  

# Combine output
wc_df = pd.concat(df_list, axis=0)

#  Annual means for each site
wc_df['year'] = wc_df['sample_date'].dt.year
an_df = wc_df.groupby(['year', 'station_id']).mean()

# Reset index and join to stns
an_df.columns = [i.split('_')[0] for i in an_df.columns]
an_df.reset_index(inplace=True)
del an_df['year']

# Join and drop NaN
wc_df = pd.merge(stn_df[['station_id', 'latitude', 'longitude']],
                 an_df, how='left', on='station_id')
wc_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Set negative values to zero
wc_df[wc_df < 0] = 0

wc_df.head()

## 2. Geology

My initial idea of downloading the NGU 1:250k geology dataset has been abandoned, because it is only possible to download small regions at a time from the NGU website. This is frustrating, but as I only need a very simple representation of the geology I'll simply switch to the coarser [Global Lithological Map of the World](https://www.geo.uni-hamburg.de/en/geologie/forschung/geochemie/glim.html) instead.

I have clipped this feature class to Fennoscandia using the shapefile here:

C:\Data\James_Work\Useful_Stuff\GIS_Data\Norway\Shapefiles\Fennoscandia.shp

### 2.1. Add geology data to PostGIS

In [ ]:
def create_wkt_element(geom, srid):
    """ Use Geoalchemy to convert Geopandas obsjects into
        WKT elements.
    """
    return WKTElement(geom.wkt, srid=srid)

In [ ]:
# Read geol shapefile
in_shp = (r'C:\Data\James_Work\Useful_Stuff\GIS_Data\Norway'
          r'\Shapefiles\GLiM_Global_Geol_Fennoscandia.shp')
gdf = gpd.read_file(in_shp)

# Convert from UTM Z33N WGS84 to UTM Z33N ETRS89
gdf = gdf.to_crs(epsg='25833')

# Explode multipolys to polys (need to join attribute table back afterwards)
exp_gdf = gdf.explode().reset_index()

# Get properties as dataframe
df = pd.DataFrame(gdf.reset_index()[['index', 'xx']])

# Join props to exploded polys
gdf = exp_gdf.merge(df, how='left', left_on='level_0', right_on='index')
gdf = gdf[[0, 'xx']]
gdf.columns = ['geom', 'lith']
gdf.set_geometry('geom')

# Calc geom col for PostGIS
gdf['geom'] = gdf['geom'].apply(create_wkt_element, args=(25833,))

gdf.to_sql('fenno_geol', schema='nope_metals', con=pg_eng,
           if_exists='replace', index=True, 
           dtype={'geom':Geometry('POLYGON', srid=25833)})

# Build spatial index
sql = ("CREATE INDEX nope_metals_fenno_geol_gix "
       "ON niva_work.nope_metals.fenno_geol "
       "USING GIST (geom)")
res = pg_eng.execute(sql)

### 2.2. Intersect geology and catchment data

The code below calculates the intersection between the geology dataset and the lake catchments.

In [ ]:
# Extract catchments 
sql = ("SELECT geom2 AS geom "
       "FROM niva_work.nope_metals.ecco_biwa "
       "WHERE nation = 'Norway'")
crs = {'init': 'epsg:25833'} # UTM Zone 33N ETRS89
cat_gdf = gpd.GeoDataFrame.from_postgis(sql, geom_col='geom', 
                                        crs=crs, con=pg_eng)

# Extract geol
sql = ("SELECT geom, xx "
       "FROM niva_work.nope_metals.fenno_geol")
crs = {'init': 'epsg:25833'} # UTM Zone 33N ETRS89
geo_gdf = gpd.GeoDataFrame.from_postgis(sql, geom_col='geom', 
                                        crs=crs, con=pg_eng)

In [ ]:
# Intersection
int_gdf = rid.spatial_overlays(geo_gdf, cat_gdf, how='intersection')

del int_gdf['geom_1'], int_gdf['geom_2']
del int_gdf['idx1'], int_gdf['idx2']

# Save
out_shp = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\NOPE\Metals\Raw_Datasets\GIS\lakes_1k_geol_inter.shp')
int_gdf.to_file(out_shp)